# HSC score prediction
## Xiaonan Wang
## 30Apr2020

In [1]:
import pandas as pd
import numpy as np
import pickle
import sklearn
import platform
import scanpy as sc
sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_versions()
sc.settings.set_figure_params(dpi=80, color_map='viridis', vector_friendly=False,  dpi_save=300)

/home/xw251/miniconda3/envs/scanpy_py368/lib/python3.6/site-packages/anndata/_core/anndata.py:21: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import RangeIndex


scanpy==1.4.6 anndata==0.7.1 umap==0.4.1 numpy==1.18.2 scipy==1.4.1 pandas==1.0.3 scikit-learn==0.22.2.post1 statsmodels==0.11.1 python-igraph==0.8.0 louvain==0.6.1


In [2]:
adata = sc.read('./write/Patel_raw_afterQC.h5ad')

In [3]:
def total_count_normalise(count_matrix):
    """Normalise count matrix for input into hscScore model.
    Performs read depth normalisation normalising each cell so that normalised 
    counts sum to the same value.
    
    Parameters
    ----------
    count_matrix : pandas dataframe
        Gene count matrix of dimension cells x genes with column names as genes
        and index as cell names
    
    Returns
    -------
    **norm_matrix** : pandas dataframe
        Normalised count matrix of dimension cells x genes
    """
    
    # Set the value normalised counts will sum to for each cell
    wilson_molo_genes_median_counts = 18704.5
    
    # Scale rows
    count_matrix_expression = np.array(count_matrix, dtype='float')
    counts_per_cell = np.sum(count_matrix_expression, axis=1)
    counts_per_cell += (counts_per_cell == 0)
    counts_per_cell /= wilson_molo_genes_median_counts
    norm_matrix_expression =  count_matrix_expression/counts_per_cell[:, None]
    norm_matrix = pd.DataFrame(norm_matrix_expression, index=count_matrix.index,
                               columns=count_matrix.columns)
    # log + 1 transform the data
    norm_matrix = np.log(norm_matrix + 1)
    
    return norm_matrix

In [4]:
# read in what's required for the model prediction, the model and the list of genes that used to train the model
hsc_score = pickle.load(open('/home/xw251/rds/rds-bg200-hphi-gottgens//users/xw251/Files/HSCscore_files/' + 'hscScore_model.pkl', 'rb'))
model_genes = np.genfromtxt('/home/xw251/rds/rds-bg200-hphi-gottgens//users/xw251/Files/HSCscore_files/' + 'model_molo_genes.txt', dtype='str')

/home/xw251/miniconda3/envs/scanpy_py368/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.preprocessing.data module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.preprocessing. Anything that cannot be imported from sklearn.preprocessing is now part of the private API.
  warnings.warn(message, FutureWarning)
/home/xw251/miniconda3/envs/scanpy_py368/lib/python3.6/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 0.21.2 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/xw251/miniconda3/envs/scanpy_py368/lib/python3.6/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neural_network.multilayer_perceptron module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding c

In [5]:
print(len(model_genes))
print(len(adata.var_names))
print(len(np.intersect1d(model_genes, adata.var_names)))

103
19806
99


In [6]:
OLgenes = np.intersect1d(model_genes, adata.var_names)

In [7]:
adata_sub = adata[:,OLgenes].copy()

In [8]:
count_data_molo = pd.DataFrame(adata_sub.X.todense(), index=adata_sub.obs_names, columns=adata_sub.var_names)

In [9]:
count_data_molo.shape

(24599, 99)

In [10]:
# add in the empty genes
missingGenes = np.setdiff1d(model_genes, adata.var_names)
print(missingGenes)
for g in missingGenes:
    count_data_molo[g] = [0]*count_data_molo.shape[0]

['2810417H13Rik' 'Fam64a' 'Sgol1' 'Sqrdl']


In [15]:
count_data_molo = count_data_molo.loc[:,model_genes].copy()

In [16]:
count_data_molo.shape

(24599, 103)

In [17]:
normalised_data_molo = total_count_normalise(count_data_molo)

In [18]:
predicted_hsc_scores = hsc_score.predict(np.array(normalised_data_molo))

In [19]:
predicted_hsc_scores

array([ 0.06468229,  0.01055192,  0.05370454, ..., -0.00708   ,
        0.01739389,  0.03743835])

In [20]:
pd.DataFrame(predicted_hsc_scores, index=adata.obs_names).to_csv('HSCscore_pred.csv')